In [4]:
!pip install --upgrade --quiet  langchain langchain-core langchain-community langchain-openai

!pip install -qU langchain-openai
!pip install -qU langchain-mistralai
!pip -q install transformers -U 
!pip -q install langchainhub

In [5]:
!pip -q install -U tenacity

In [6]:

import getpass
import os
import json
from tqdm import tqdm
import pandas as pd
from iesta.evaluator import llm_ideology


from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import iesta
import iesta.evaluator.llm_ideology


import importlib

importlib.reload(iesta)
importlib.reload(iesta.evaluator.llm_ideology)


dotenv was False
dotenv was False


<module 'iesta.evaluator.llm_ideology' from '/Users/roxanneelbaff/Documents/projects/github/eacl2024-iesta/iesta/evaluator/llm_ideology.py'>

In [7]:
models = ["gpt-3.5-turbo", "meta-llama/Llama-2-7b-chat-hf"] #["gpt-4","open-mixtral-8x7b",]# "gpt-3.5-turbo", "meta-llama/Llama-2-7b-chat-hf"]
ideologies = ["None"]
prompt_types = ["role_play_prompt_4"]#, "imagine", "role_play_prompt_1", "role_play_prompt_2"]


## Preliminary experiment: Prompt 2 generation from prompt1 for role play 


In [ ]:
llm_ideology.get_prompt2_samples(
    model_lst = models,
    ideology_lst = ideologies,
    temperature = 0.7,
    repetitions = 50,
    root = "data/",
)

prompt2_top_df = llm_ideology.get_top_prompt2("data/")

## MAIN EXPERIMENT - RUN PEW Quiz


In [ ]:
%%time

import logging
import sys
from tenacity import retry, stop_after_attempt, before_log, wait_fixed

logging.basicConfig(stream=sys.stderr, level=logging.INFO)

logger = logging.getLogger(__name__)

def run_pew_all_settings():
    repititions = 30
    for prompt_type in tqdm(prompt_types):
        for model_name in tqdm(models):
            for ideology in tqdm(ideologies):
                if ideology == "None":
                    continue
                try:
                    run_tests(model_name, repititions, ideology, prompt_type, )
                except Exception as e:
                    print(model_name, ideology, prompt_type,)
                
@retry(stop=stop_after_attempt(5), wait=wait_fixed(2))#, before=before_log(logger, logging.INFO))
def run_tests(model_name, repititions, ideology, prompt_type, ):
    print("MODEL: ", model_name)
    print("IDEOLOGY: ", ideology)
    print("PROMPT TYPE: ", prompt_type)
    pew_for_llm = llm_ideology.PEWQuizForLLM(model_name,
                                            repeat_quiz = 3,
                                            ideology_key = ideology,
                                            prompt_type = prompt_type,
                                            root = "data/",
                                            temperature = 0.0)
    # 80/3 per iteration
    pew_for_llm.repeat_pew_quiz()
    print("---------- FINISHED---------------") 
    

In [ ]:
%%time

run_pew_all_settings()

In [3]:
import pandas as pd
pd.read_csv("data/liberal_llm_based_eval1-3.csv").columns.tolist()

['id',
 'ideology',
 'batch',
 'iteration',
 'prompt',
 'original_effectiveness',
 'original_clarity',
 'original_feedback',
 'rewrite1_effectiveness',
 'rewrite1_clarity',
 'rewrite1_consistency',
 'rewrite1_feedback',
 'rewrite2_effectiveness',
 'rewrite2_clarity',
 'rewrite2_consistency',
 'rewrite2_feedback',
 'favorite',
 'original_effectiveness_feedback',
 'original_clarity_feedback',
 'rewrite1_effectiveness_feedback',
 'rewrite1_clarity_feedback',
 'rewrite1_consistency_feedback',
 'rewrite2_effectiveness_feedback',
 'rewrite2_clarity_feedback',
 'rewrite2_consistency_feedback',
 'original_consistency']

In [ ]:
# 1 min 51 sec. for 4 iterations + 1 min 50s
# 10 iter: 3 59.1 s
# 6 : 2m 41.8s +  2m

# gpt 4 : 30 iterations 12.365 minutes. cost: 3.02 $


In [ ]:
import glob
pew_result_file_lst = glob.glob("data/llm_ideology/pew_quiz_results/*.csv")
#assert len(pew_result_file_lst) == 780

In [ ]:
pew_result_file_lst[0].split("_")